# Webscraping

Ce code permet de scraper des images sur le site internet Plantnet, et voici comment il procède : 

D’abord il faut importer les bonne librairies et modules :

* Selenium qui est une librairie pour le scraping
* Ses sous-modules : webdriver, By, ActionChain, WebdriverWait, expected_conditions sont des fonction communes et de soutien de webdriver. 
* request est un  module permettant de faire des requêtes
* shutil est un modèle permettant de faire des opérations sur des fichiers
* OS est un module qui fournit une façon portable d'utiliser les fonctionnalités dépendantes du système d'exploitation

Pour Scraper les images des différentes plantes choisies on crée un dictionnaire appelé ici `FLOWER_URL`, ce dictionnaire contient en clés le nom scientifique de la fleur, et en valeur son url sur le site. 

Pour télécharger on crée une fonction download_image(links) qui permet que pour chaque lien enumérer, renvoie comme réponse un `request.get` sur le lien, et permet ensuite d’ouvrir et de télécharger toutes les images du format demandé et du lien lu, puis on supprime la `response`. 

On définit aussi comme `fonction navigate_to_image(url)` : 
cette fonction permet de lire l’url de chaque fleurs, et on implémente aussi un wait pour marquer une pause afin que l’icone fleur soit visible. 
Puis on utilise une variable `flower_icon` dans laquelle ont insère le xpath de l’icône fleur, pour qu’avec la fonction click, on puisse cliquer sur l’icône en forme de fleur. 

![imagefleur.png](attachment:imagefleur.png)

On implémente un autre wait d’une minute 15 afin que toutes les images soient disponibles
Quand le délai est écoulé le code qui suit permet de capturer les élément web ayant une prévisualisation d’image et prend les liens. 
On utilise pour sa une variable qui stockera les lien obtenus via css_selector. 
On demande ensuite un return du src pour les fleur dans la variable flowers. 

Après avoir écrit ces 2 fonctions, on définit le navigateur (ici Firefox) via : 
browser = webdriver.Firefox(path_du_geckodriver)

il faut maintenant itérer chacun des noms et des liens : pour cela la boucle for est utilisé, afin que le nom, et l’url à l’intérieur du dictionnaire soit récupéré via .items qui permet de parcourir les clés et les valeur  d’un dictionnaire, avec name qui contiendra les clés du dictionnaire FLOWER_URL et url les url du dictionnaire. 

On créer en restant dans la boucle for un nouveau répertoire pour stocker les image s’il n’existe pas. 
Une variable dirname est crée et utilisé, afin de remplacer dans les training_data/raw les " " par des "_" et les "." par des "", pour créer de nouveau path. 
Si il n’existe pas de path du dirname, alors un nouveau répertoire sera crée a partir de dirname. 
La variable `img_links` stockera les src de chaque image de l’url, et chaque image sera téléchargé a partir des donné stocké dans `img_links`

Si le path du répertoire existe déjà : alors on applique directement la fonction `navigate_to_image(url)` en l’appelant avec `img_links(comme précédemment)`. 

On rajoute une condition : si la longueur de la liste de tous les fichiers du répertoire, est inférieure à la longueur de la liste `img_links`, cela signifie que `img_links = img_links`  dans dirname. 

Ont télécharge ensuite les images, puis après avoir quitté la condition on ferme le navigateur avec `browser.close`. 

In [ ]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import shutil
import requests
import os

"""
Scraping du site : https://identify.plantnet.org/explo/salad/

On s'intéresse aux espèces stockées dans la variable FLOWER_URL. Le script
va prendre chacune de ces URL et va stocker toutes les images de prévisualisation
dans le dossier correspondant.
"""


FLOWER_URL = {
    'Campanula persicifolia L.': 'https://identify.plantnet.org/species/salad/Campanula%20persicifolia%20L.',
	'Cichorium intybus L.': 'https://identify.plantnet.org/species/salad/Cichorium%20intybus%20L.' ,
    'Lepidium draba L.' :'https://identify.plantnet.org/species/salad/Lepidium%20draba%20L.',
    'Leucanthemum vulgare (Vaill.) Lam.' : 'https://identify.plantnet.org/species/salad/Leucanthemum%20vulgare%20(Vaill.)%20Lam.',
    'Malva sylvestris L.': 'https://identify.plantnet.org/species/salad/Malva%20sylvestris%20L.'
}

#Télécharge les images dans le dossier
def download_image(links):
    for i, link in enumerate(links):
        response = requests.get(link, stream=True)
        with open(f'{dirname}/img_{i}.jpg', 'wb') as out_file:
            shutil.copyfileobj(response.raw, out_file)
        del response

# Navigue sur la page URL et récupère les images dessus
def navigate_to_images(url):
    browser.get(url)
    #Marque une pause pour que l'élément en forme d'icône de fleur soit visible
    pause = WebDriverWait(browser, 10).until(EC.visibility_of_element_located((By.XPATH, "/html/body/div/div/div/div[2]/ul/li[2]/a/img")))
    #Clique sur l'icône fleur pour afficher les images en forme de fleur
    flower_icon = browser.find_element_by_xpath('/html/body/div/div/div/div[2]/ul/li[2]/a/img')
    flower_icon.click()

    #Marque une pause de 1:15 maximum pour que toutes les images soient disponibles
    pause = WebDriverWait(browser, 75).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR , "img.img.img-responsive.img-thumbnail")))
    
    #Capture tous les éléments web ayant une prévisualisation d'image et prend les liens
    flowers = browser.find_elements_by_css_selector("img.img.img-responsive.img-thumbnail")
    return [flower.get_attribute("src") for flower in flowers]

# Définit le navigateur sur Firefox par défaut
browser = webdriver.Firefox()

#Itère sur chacun des noms et des liens
for name, url in FLOWER_URL.items():

    #Crée un nouveau répertoire pour stocker les images si il n'existe pas
    dirname = "../training_data/raw/" + name.replace(" ", "_").replace(".","")
    if not os.path.exists(dirname):
        os.makedirs(dirname)

        img_links = navigate_to_images(url)

        download_image(img_links)
    else:
        img_links = navigate_to_images(url)
        #Vérifie si toutes les images sont là, sinon itère sur le reste
        if len(os.listdir(dirname)) < len(img_links):
            img_links = img_links[len(os.listdir(dirname)):]
            download_image(img_links)

# ferme le navigateur
browser.close()